IMPORT


if you want want to use the notebook in google colabs you can use the code below, but if you want to use it locally you can skip the first step to mount the google drive.

In [129]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

petroleum data scraped from https://www.energy.gov.za/files/esources/petroleum/petroleum_arch.html

In [139]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor

from lightgbm import LGBMRegressor

import math
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000

#put the your data folder path here
path = '/content/drive/MyDrive/submit'
cpi = pd.read_csv('CPI_Historic_Values_Zindi_May_23.csv')
petrol = pd.read_csv('petrolem.csv')
seed = 3

PIVOTING THE DATA


In [131]:
cpi_pivot = cpi.pivot(index = 'Month', columns = 'Category', values = 'Value').reset_index()#changing from a long format to a wide format,
cpi_pivot['Month'] = pd.to_datetime(cpi_pivot['Month'])
cpi_pivot = cpi_pivot.sort_values("Month").reset_index(drop=True)

ADDING JUNE'S DATA MANUALLY FROM https://www.statssa.gov.za/publications/P0141/P0141June2023.pdf


In [132]:
date_str = '2023-06-30'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)
cpi_of_june = [110.9, 104.3, 99.6, 110.4, 118.3, 109.8, 110.8, 107.7, 105.4, 109.6, 105.3, 110.0, 112.3]

for i, col in enumerate(cpi_pivot.columns[1:]):
    cpi_pivot.at[17, col] = cpi_of_june[i]

ADD THE july ROW


In [133]:
date_str = '2023-07-31'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)

FEATURE ENGINNERING


the only feature i used is a 5 times lagging features of all the month's cpi's. nothing fancy other than that.

In [134]:
feats_to_lag = cpi_pivot.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,5):
        cpi_pivot[f'prev_{i}_month_{col}'] = cpi_pivot[col].shift(i)
columns = list(cpi_pivot.columns)
columns = [item for item in columns if "Transport" not in item]

Add petrol data

In [135]:
petrol['Month'] = pd.to_datetime(petrol['Month'])
petrol = petrol.sort_values("Month").reset_index(drop=True)
feats_to_lag = petrol.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,7):
        petrol[f'prev_{i}_month_{col}'] = petrol[col].shift(i)
start_date = pd.to_datetime('2022-01-31')
end_date = pd.to_datetime('2023-07-31')
petrol = petrol[(petrol['Month'] >= start_date) & (petrol['Month'] <= end_date)]
cpi_pivot = pd.merge(cpi_pivot, petrol, on='Month', suffixes=('_df1', '_df2'))

HANDLE MISSING DATA


In [136]:
cpi_pivot = cpi_pivot.drop(0)
cpi_pivot = cpi_pivot.bfill()

TRAIN AND VALIDATION


In [137]:
train = cpi_pivot[cpi_pivot['Month'] != "2023-07-31"]
test = cpi_pivot[cpi_pivot['Month'] == "2023-07-31"]

training_set = train[train['Month']!= '2023-06-30']
validation_set = train[train['Month']== '2023-06-30']

train.shape, test.shape, training_set.shape, validation_set.shape

((17, 108), (1, 108), (16, 108), (1, 108))

MODELING


In [151]:
target_cols = ['Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Communication', 'Education', 'Food and non-alcoholic beverages',
       'Headline_CPI', 'Health', 'Household contents and services',
       'Housing and utilities', 'Miscellaneous goods and services',
       'Recreation and culture', 'Restaurants and hotels ', 'Transport']

#if you add additional data sources that have no value in the predicting month , drop it, now that you have their lags
features= [col for col in train.columns if col not in target_cols + ['Month']]
featurest= [col for col in train.columns if col not in columns + ['Transport']]

X_train = training_set[features]
X_traint = training_set[featurest]
y_train = training_set[target_cols]

X_val = validation_set[features]
X_valt = validation_set[featurest]
y_val = validation_set[target_cols]

l_models = {}

y_predl = []

rmsel_dict = {} 

scaler = MinMaxScaler()

#training
for target_col in target_cols:
    if target_col == "Transport":
        #l_model = Lasso(alpha=0.1, random_state=seed)
        l_model = XGBRegressor(seed=seed)
        X_train_scaled = scaler.fit_transform(X_traint)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_valt)

        y_pred_coll = l_model.predict(X_val_scaled)

        rmsel_col = np.sqrt(mean_squared_error(y_pred_coll, y_val[target_col]))
        rmsel_dict[target_col] = rmsel_col 

        y_predl.append(y_pred_coll)
    else:
        l_model = Lasso(alpha=0.1, random_state=seed)
        X_train_scaled = scaler.fit_transform(X_train)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_val)

        y_pred_coll = l_model.predict(X_val_scaled)

        rmsel_col = np.sqrt(mean_squared_error(y_pred_coll, y_val[target_col]))
        rmsel_dict[target_col] = rmsel_col 

        y_predl.append(y_pred_coll)

# scoring
y_predl = np.array(y_predl).T

dfl = pd.DataFrame({'y_pred': y_predl.flatten(), 'y_val': y_val.values.flatten()})

# Print RMSE for each target column
for target_col in target_cols:
    print(f'RMSE of Lasso Regression for {target_col}: {rmsel_dict[target_col]}')

# Calculate the average RMSE across all target columns
average_rmse = np.mean(list(rmsel_dict.values()))
print(f'Average RMSE of Lasso Regression: {average_rmse}')

RMSE of Lasso Regression for Alcoholic beverages and tobacco: 0.30152740478516193
RMSE of Lasso Regression for Clothing and footwear: 0.20145874023437216
RMSE of Lasso Regression for Communication: 0.13010253906250568
RMSE of Lasso Regression for Education: 0.0005172729492244343
RMSE of Lasso Regression for Food and non-alcoholic beverages: 0.6013153076171847
RMSE of Lasso Regression for Headline_CPI: 0.3680618286132784
RMSE of Lasso Regression for Health: 0.7155197143554659
RMSE of Lasso Regression for Household contents and services: 0.2961837768554716
RMSE of Lasso Regression for Housing and utilities: 0.8009933471679744
RMSE of Lasso Regression for Miscellaneous goods and services: 0.20134124755858807
RMSE of Lasso Regression for Recreation and culture: 0.4021804809570284
RMSE of Lasso Regression for Restaurants and hotels : 0.20745086669921875
RMSE of Lasso Regression for Transport: 0.1844223022460909
Average RMSE of Lasso Regression: 0.3393134483924281


In [152]:
X_train = train[features]
X_traint = train[featurest]
y_train = train[target_cols]

X_val = test[features]
X_valt = test[featurest]
y_val = test[target_cols]

l_models = {}

y_predl = []

scaler = MinMaxScaler()

#training
for target_col in target_cols:
    if target_col == "Transport":
        #l_model = Lasso(alpha=0.1, random_state=seed)
        l_model = XGBRegressor(seed=seed)
        X_train_scaled = scaler.fit_transform(X_traint)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_valt)

        y_pred_coll = l_model.predict(X_val_scaled)

        y_predl.append(y_pred_coll)
    else:
        l_model = Lasso(alpha=0.1, random_state=seed)

        X_train_scaled = scaler.fit_transform(X_train)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_val)

        y_pred_coll = l_model.predict(X_val_scaled)

        y_predl.append(y_pred_coll)

july = [111.5, 104.5, 99.5, 110.4, 118.5, 110.8, 110.6, 108.2 , 108.4, 109.9, 105.5, 110, 112.6]
y_predl = np.array(y_predl).T
july = np.array(july).T

#print(f'prediction of Lasso Regression: {y_predl}')
rmse_values = {}

for i, target_col in enumerate(target_cols):
    y_pred_col = y_predl[:, i][0] # Predictions for the current target column
    july_col = july[i]
    rmse_col = y_pred_col - july_col 
    rmse_values[target_col] = rmse_col  # Store RMSE value in the dictionary

    print(f'RMSE for {target_col}: {rmse_col}')
july = [[111.5], [104.5], [99.5], [110.4], [118.5], [110.8], [110.6], [108.2] , [108.4], [109.9], [105.5], [110], [112.6]]
july = np.array(july).T
# Calculate the average RMSE across all target columns
average_rmse = np.sqrt(mean_squared_error(y_predl, july))
print(f'Average RMSE for all columns: {average_rmse}')

RMSE for Alcoholic beverages and tobacco: -0.6012954711914062
RMSE for Clothing and footwear: -0.20121002197265625
RMSE for Communication: 0.18988800048828125
RMSE for Education: -0.00044097900391193434
RMSE for Food and non-alcoholic beverages: -0.20128631591796875
RMSE for Headline_CPI: -1.0020568847656222
RMSE for Health: 0.19883575439453693
RMSE for Household contents and services: -0.5058700561523466
RMSE for Housing and utilities: -3.0012268066406307
RMSE for Miscellaneous goods and services: -0.30181732177734943
RMSE for Recreation and culture: -0.2012939453125
RMSE for Restaurants and hotels : 0.39847564697265625
RMSE for Transport: -0.32411346435546307
Average RMSE for all columns: 0.9274025508864118


SUBMITION


In [ ]:
def prepSub(y_pred:list, target_cols: list, test, prefix:str):
    sub_df = pd.DataFrame(y_pred, columns=target_cols)
    sub_df['Month'] = test['Month']

    sub_df.set_index('Month', inplace=True)
    sub_df.columns = [prefix+'_' + col.lower().replace('_', ' ').strip() for col in sub_df.columns]
    sub_df.rename(columns= {f"{prefix}_headline cpi": f"{prefix}_headline CPI"}, inplace=True)

    sub_df = pd.melt(sub_df.reset_index(), id_vars= ['Month'], var_name= 'ID', value_name= 'Value')

    return sub_df[['ID', 'Value']]

In [ ]:
sub = prepSub(y_predl, target_cols, test, 'July')
sub.to_csv(f'out/lasso_with_lag5_historic_petroledl7.csv', index=False)